In [2]:
import jieba
#import jieba.analyse

import gensim
import pickle
import os
from itertools import chain
from time import strftime, localtime
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer, HashingVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

import matplotlib.pyplot as plt 

from sklearn.pipeline import Pipeline
import joblib

from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive')
from NLP_function import *
os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Run') 

##### * parameters *

In [3]:
file = 'C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Data_Prepare/Project_pogress_20180509.xlsx'
sampid = ['A2023', 'A2042','A2108','A2418','A2172','A4305','A2769',
         'A3455','A3450','A3439','A3435','A3444','A4304','A3691','A32940','A2023',
         'A0000','A4537','A4222','A4382','A4442','A3581','A4583','A0000']
#sampid = ['A2023']
lables_hpo = pd.read_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn/lables_hpo.csv', encoding="ANSI")

In [4]:
lables_hpo

,lable,id,lable1
0,1型糖尿病,HP:0100651,1型糖尿病
1,Abnormal zona pellucida morphology,HP:0020156,Abnormalzonapellucidamorphology
2,Cervical neoplasm,HP:0032241,Cervicalneoplasm
3,DNA修复异常,HP:0003254,DNA修复异常
4,Hydatidiform mole,HP:0032192,Hydatidiformmole
...,...,...,...
452,自闭症行为,HP:0000729,自闭症行为
453,自然流产,HP:0005268,自然流产
454,自然流产;胚胎发病;严重的胎儿宫内发育迟缓,HP:0005268;HP:0011460;HP:0008846,自然流产胚胎发病严重的胎儿宫内发育迟缓
455,足多趾,HP:0001829,足多趾


#### Data import _ use original clinical data

In [23]:
# use original clinical data
df = pd.read_excel(file)
df = df.iloc[:, np.r_[0, 5, 17, 21:58]]
df = df.loc[df['样本编号'].isin(sampid)]

# merge title and content
for a in range(0,df.shape[1]-1):
    if not a in np.r_[0,1,2,3,7:19,22:df.shape[1]]:
        i = df.columns[a]
        df[i] = df[i].apply(lambda x : str(x) if pd.isnull(x) else i.replace('精阅体格检查\n/','') + str(x) )
        df[i] = df[i].apply(lambda x : x.replace('伊阅',''))  
        df[i] = df[i].apply(lambda x : x.replace('nan', ''))  
        #df = df[~df[i].isin([''])]        
        
# change number into chinese character 
#col = chain(range(3,8),range(9,16), range(29,35)) # for several panel
col = range(3,40)                                  # for all
for c in col:
    df.iloc[:,c] = df.iloc[:,c].apply(lambda x : change_number(x,'cn'))
    
df = df.drop_duplicates()


# transform into list and remove float phenotype
df = df.T
df.columns = df.iloc[0]
res = []
for i in range(0,df.shape[1]):
    res.append(pd.Series(df.iloc[0:,i]).dropna().drop_duplicates().values) 
for i in range(0,len(res)):
    res[i] = [x for x in res[i] if type(x) != float] 

    
# sentence tokenization
for a in range (0,len(res)):
    for b in range(0,len(res[a])):
        #print(res[a][b])
        res[a][b] = re.split('[，]|[,]|[；]|[：]|[。]', res[a][b])
        #print(res[a][b])      
# save and export to numpy array
#np.save('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Run/cli_tokenizing_res.npy', np.array(res))

In [24]:
# read hpo data
hpo = pd.read_csv('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/hpo_import_python/Hpo_cn_clean.csv')


### extract information
txt = pd.DataFrame(columns=['id', 'pheno'])

for i in range(0,len(res)):
    length = len(list(chain(*res[i][2:len(res[i])])))
    tem = pd.DataFrame({'id': pd.DataFrame(res[i][0]).values.repeat(length), 'pheno': list(chain(*res[i][2:len(res[i])])) })
    txt = txt.append(tem)
txt['pheno'] = txt['pheno'].dropna()
txt = txt.drop_duplicates()
txt = txt.reset_index(drop=True)


### clean txt (remove non sense doc)
searchfor = ["正常",'零次']

In [25]:
txt['pheno'] = txt['pheno'].apply(lambda x : clean_text(x, lan='cn'))
txt['pheno'] = txt['pheno'].dropna()
txt= txt[txt.pheno!='nan']

txt = txt[~txt.pheno.str.contains('|'.join(searchfor), na=False)] # remove str contraing specific str

txt = txt[~txt.pheno.isin(['', '.', '重点关注','无','既往病史','全外扫描','未查','B超','其他遗传学检查结果','其他检查','要求全外扫描'])] 
txt['pheno'] = txt['pheno'].apply(lambda x: x.replace('45,X','45_X'))

txt = txt.reset_index(drop=True)


### jieba cut
txt['pheno_cut'] = txt['pheno'].apply(lambda txt: list(jieba.cut(txt))) #调用结巴分词

In [26]:
txt['pheno']

0              精囊腺所含凝固因子不足
1                    具体表现为
2       前列腺产生的蛋白分解酶溶纤蛋白酶不足
3                 精索静脉曲张明显
4                     疑似炎症
              ...         
137                     生化
138           二零一七年六月移植未着床
139    二零一九年五月冷冻胚胎移植移植一枚D三
140                    无胎心
141                   四六XX
Name: pheno, Length: 142, dtype: object

In [ ]:
res

In [16]:
txt.iloc[0:20,:]

,id,pheno,pheno_cut
0,A0000,精囊腺所含凝固因子不足,"[精囊, 腺所含, 凝固, 因子, 不足]"
1,A0000,具体表现为,"[具体表现, 为]"
2,A0000,前列腺产生的蛋白分解酶溶纤蛋白酶不足精索静脉曲张明显,"[前列腺, 产生, 的, 蛋白, 分解, 酶, 溶纤, 蛋白酶, 不足, 精索, 静脉曲张,..."
3,A0000,疑似炎症,"[疑似, 炎症]"
4,A0000,睾酮低,"[睾酮, 低]"
5,A0000,死精子数略高临床初步判断为精液不液化症,"[死, 精子, 数略, 高, 临床, 初步判断, 为, 精液, 不, 液化, 症]"
6,A4537,左上腹疼痛一年余,"[左, 上腹, 疼痛, 一年, 余]"
7,A4537,二零一九年七月一一日行肠镜下结肠息肉电切术,"[二零一, 九年, 七月, 一, 一日, 行肠镜, 下, 结肠, 息肉, 电切术]"
8,A4537,结肠多发息肉,"[结肠, 多发, 息肉]"
9,A4537,无家族史,"[无, 家族史]"


In [11]:
%store txt
del txt

Stored 'txt' (DataFrame)


In [11]:
### Prediction _ skl ： load model and tokenizer
with open('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn/tokenizer_keras_cn_skl.pickle', 'rb') as handle:
    skl_tokenizer = pickle.load(handle)
with open('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn/M_svm_classifier_cn.pickle', 'rb') as handle:
    skl = pickle.load(handle)
    
x = txt['pheno_cut']
X = skl_tokenizer.texts_to_matrix(x, mode='tfidf')
txt['pred_lable_skl'] = skl.predict(X)
txt['pred_id_skl'] = txt['id']


### Prediction _ mlp ： load model and tokenizer
with open('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn/tokenizer_keras_cn_skl.pickle', 'rb') as handle:
    skl_tokenizer = pickle.load(handle)
with open('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn/M_mlp_classifier_cn.pickle', 'rb') as handle:
    mlp = pickle.load(handle)
    
x = txt['pheno_cut']
X = skl_tokenizer.texts_to_matrix(x, mode='tfidf')
txt['pred_lable_mlp'] = mlp.predict(X)
txt['pred_id_mlp'] = txt['id']

In [13]:
### Prediction _ nn ： load model and tokenizer
model = tf.keras.models.load_model('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn/M_bow_neuralnetwork_cn.h5') # or Conv1D_model.h5
print('Load model...')
#model.summary()
with open('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn/tokenizer_keras_cn_nn.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
    
lables_nn = pd.read_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn/lables_nn.csv', encoding="ANSI")

txt['pred_lable_nn'] = txt['id']
txt['pred_id_nn'] = txt['id']

for i in range(0,len(txt)):
    x = txt['pheno_cut'][i]
    X = loaded_tokenizer.texts_to_matrix([x], mode='tfidf')
    pred = model.predict_classes(X)
    txt['pred_lable_nn'][i] = pd.Series(lables_nn[lables_nn['y_id']==pred[0]]['y_labels'].reset_index(drop=True))[0]
  



### Remove repeat values and export predict results
for i in range(0,len(txt)):
    txt['pred_id_skl'][i] = pd.Series(lables_hpo[lables_hpo['lable']==txt['pred_lable_skl'][i]]['id'].reset_index(drop=True))[0]
    txt['pred_id_mlp'][i] = pd.Series(lables_hpo[lables_hpo['lable']==txt['pred_lable_mlp'][i]]['id'].reset_index(drop=True))[0]
    txt['pred_id_nn'][i]  = pd.Series(lables_hpo[lables_hpo['lable']==txt['pred_lable_nn'][i]]['id'].reset_index(drop=True))[0]

    
### transfrom into list
for c in range(3,txt.shape[1]):
    txt.iloc[:,c] = txt.iloc[:,c] .str.split(';').tolist()  
    
    

    
res = txt.groupby(['id'])['pheno','pred_lable_skl','pred_id_skl','pred_lable_mlp','pred_id_mlp','pred_lable_nn','pred_id_nn',].agg(lambda x: list(x)).reset_index(drop=False)

### convert nested list into a one-dimensional list
for i in range(0,res.shape[0]):
    for c in range(2, res.shape[1]):
        res.iloc[:,c][i] = list(flatten(res.iloc[:,c][i]))
        
for i in res.columns[1:res.shape[1]]:
    res[i] = res[i].apply(lambda x : list(set(x)))
    res[i].apply(lambda x : x.remove('无') if x.count('无') !=0  else x)
    
    
    

### search common terms from two models
res['pred_lable_common'] = res['pred_id_common'] = res['id']
for i in range(0,len(res)):
    res['pred_lable_common'][i] = list(set(list(set(list(set(res['pred_lable_mlp'][i])&set(res['pred_lable_nn'][i])))))&set(res['pred_lable_skl'][i]))
    res['pred_id_common'][i] = list(set(list(set(list(set(res['pred_id_mlp'][i])&set(res['pred_id_nn'][i])))))&set(res['pred_id_skl'][i]))
    
    
res = res[['id','pheno','pred_lable_common','pred_id_common' ,'pred_lable_skl','pred_id_skl','pred_lable_mlp','pred_id_mlp','pred_lable_nn','pred_id_nn']]
res.to_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Run/prediction_sup_'+ strftime("%Y%m%d_%H%M", localtime()) +'.csv', header=True, index=False, encoding="ANSI")
res

Load model...
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


<ipython-input-13-f6cb1b061d7b>:36: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  res = txt.groupby(['id'])['pheno','pred_lable_skl','pred_id_skl','pred_lable_mlp','pred_id_mlp','pred_lable_nn','pred_id_nn',].agg(lambda x: list(x)).reset_index(drop=False)


,id,pheno,pred_lable_common,pred_id_common,pred_lable_skl,pred_id_skl,pred_lable_mlp,pred_id_mlp,pred_lable_nn,pred_id_nn
0,A2023,"[六九XXY, 重点关注葡萄胎相关基因, 四六XX, 复发性葡萄胎, 葡萄胎二次, 第二次流...","[染色体分离异常, Hydatidiform mole]","[HP:0002916, HP:0032192]","[染色体分离异常, Hydatidiform mole, 性反转]","[HP:0012245, HP:0002916, HP:0032192]","[染色体分离异常, 染色体稳定性异常, Hydatidiform mole]","[HP:0003220, HP:0002916, HP:0032192]","[染色体分离异常, 染色体稳定性异常, Hydatidiform mole]","[HP:0003220, HP:0002916, HP:0032192]"
